In [1]:
# 붓꽃 품종 예측하기
from sklearn.datasets import load_iris # 사용할 데이터
from sklearn.tree import DecisionTreeClassifier # 사용할 알고리즘
from sklearn.model_selection import train_test_split # 학습데이터와 테스트데이터를 분리하는 함수

In [2]:
import pandas as pd
iris = load_iris()
iris_data = iris.data # 피처만으로 된 데이터
iris_label = iris.target # 레이블(결정값) 데이터

iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
# 학습용 데이터와 테스트용 데이터 분리 train_test_split()
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label, test_size=0.2, random_state=11)
                                                    #(피처 데이터, 레이블 데이터, 테스트데이터 비율, 난수 발생값 (seed와 같은 의미))

In [4]:
# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

# 학습 수행
dt_clf.fit(X_train, y_train)

# 테스트 데이터 세트로 예측 수행
pred = dt_clf.predict(X_test)

In [5]:
pred

array([2, 2, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0,
       0, 1, 0, 0, 2, 1, 0, 1])

In [7]:
y_test

array([2, 2, 2, 1, 2, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 0,
       0, 1, 0, 0, 2, 1, 0, 1])

In [13]:
predict_result = pd.DataFrame(pred , columns=['pred'])
predict_result['y_test'] = y_test
predict_result

,pred,y_test
0,2,2
1,2,2
2,1,2
3,1,1
4,2,2
5,0,0
6,1,1
7,0,0
8,0,0
9,1,1


In [5]:
# 정확도 측정
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9333


In [14]:
# 정확도 측정
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(pred, y_test)))

예측 정확도: 0.9333


고정된 학습 데이터와 테스트 데이터로 평가를 하다 보면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 생기게 된다.

결국은 해당 테스트 데이터에만 과적합되는 학습 모델이 만들어져 다른 테스트용 데이터가 들어올 경우에는 성능이 저하된다.

이러한 문제점을 개선하기 위해 교차 검증을 이용해 더 다양한 학습과 평가를 수행한다.

### 교차 검증

In [6]:
# 과적합: 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에 예측 성능이 과도하게 떨어지는 것
# 교차 검증은 이러한 데이터 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행함

#### k폴드 교차 검증

In [7]:
# K=5의 경우, 총 5개의 폴드 세트에 5번의 학습과 검증 평가 반복 수행
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

kfold = KFold(n_splits = 5) # 5개의 폴드 세트로 분리
cv_accuracy = [] # 폴드 세트별 정확도를 담을 리스트

print('붓꽃 데이터 세트 크기:', features.shape)

붓꽃 데이터 세트 크기: (150, 4)


In [8]:
# 150개의 데이터이므로 학습용 데이터 세트는 4/5인 120개, 검증 데이터 세트는 1/5인 30개)

n_iter = 0 

for train_index, test_index in kfold.split(features): # kfold 객체의 split()을 호출하면 폴드 별 학습용, 검증용 데이터의
                                                        # 로우 인덱스를 array로 반환

        X_train, X_test = features[train_index], features[test_index] # 학습용, 검증용 테스트 데이터 추출
        y_train, y_test = label[train_index], label[test_index]
        
        dt_clf.fit(X_train, y_train)
        pred = dt_clf.predict(X_test)
        
        n_iter +=1
        
        accuracy = np.round(accuracy_score(y_test, pred), 4) # np.round(data, 넷째자리로 반올림)
        train_size = X_train.shape[0]
        test_size = X_test.shape[0]
        
        print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기:{2}, 검증 데이터 크기: {3}'
              .format(n_iter, accuracy, train_size, test_index))
        print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
        cv_accuracy.append(accuracy)
        
print('\n## 평균 검증 정확도:',np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기:120, 검증 데이터 크기: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기:120, 검증 데이터 크기: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기:120, 검증 데이터 크기: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기:120, 검증 데이터 크기: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 1

#### Strarified K 폴드

In [9]:
# KFold의 문제점: 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못한다.
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [10]:
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [11]:
kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [12]:
# StratifiedKFold는 레이블 데이터 세트가 전체 레이블 값의 분포도를 반영하게 한다.

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']): # 데이터 세트 뿐만 아니라 레이블 데이터 세트도 반드시 필요
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


In [20]:
df_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
        X_train, X_test = features[train_index], features[test_index] # 학습용, 검증용 테스트 데이터 추출
        y_train, y_test = label[train_index], label[test_index]
        
        dt_clf.fit(X_train, y_train)
        pred = dt_clf.predict(X_test)
        
        n_iter +=1
        
        accuracy = np.round(accuracy_score(y_test, pred), 4) # np.round(data, 넷째자리로 반올림)
        train_size = X_train.shape[0]
        test_size = X_test.shape[0]
        
        print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기:{2}, 검증 데이터 크기: {3}'
              .format(n_iter, accuracy, train_size, test_index))
        print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
        cv_accuracy.append(accuracy)

print('\n## 교차 검증별 정확도:',np.round(cv_accuracy, 4))
print('\n## 평균 검증 정확도:',np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.98, 학습 데이터 크기:100, 검증 데이터 크기: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기:100, 검증 데이터 크기: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
#2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기:100, 검증 데이터 크기: [ 34  35  36  3

In [21]:
# 따라서 분류에서의 교차검증은 K폴드가 아니라 Stratified K폴드를 사용해야 한다.

In [28]:
# 위와 같은 교차 검증을 간편하게 - cross_val_score()
# 내부에서 Estimator를 학습(fit), 예측(predict), 평가(evaluation) 시켜준다.

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate # cross_validate()는 여러 개의 평가 지표를 반환할 수 있다.
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv=3) #(estimator, X, y=none, scoring=none, cv=none)
                                                                          #(회귀or분류, 피처, 레이블, 성능평가지표, 폴드 수)
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도:',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


In [16]:
# 교차 검증과 최적 파라미터 튜닝을 한번에 - GridSearchCV

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV , train_test_split

iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size = 0.2, random_state=121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]} 

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv = 3, refit=True) 
# refit은 디폴트가 True이며 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력한 estimartor 객체를 해당 하이퍼 파라미터로 재학습 시킨다.

grid_dtree.fit(X_train, y_train) # 학습데이터 세트를 fit 메서드에 인자로 입력
                                 # 메서드를 수행하면 그 결과를 cv_results_ 속성에 기록한다.

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [18]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00066527, 0.00098896, 0.00099738, 0.00066535, 0.00099659,
        0.00066447]),
 'std_fit_time': array([4.70415035e-04, 1.03394160e-05, 7.86741172e-07, 4.70477020e-04,
        1.66324373e-06, 4.69853077e-04]),
 'mean_score_time': array([0.00066535, 0.        , 0.        , 0.00033156, 0.00066527,
        0.0006667 ]),
 'std_score_time': array([0.00047047, 0.        , 0.        , 0.0004689 , 0.00047042,
        0.00047143]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_samples_split': 3},
  {'ma

In [20]:
# GridSearchCV의 결과를 추출해 DataFrame으로 변환
import pandas as pd
score_df = pd.DataFrame(grid_dtree.cv_results_) # 
score_df = score_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]
score_df

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [28]:
score_df.sort_values(by = 'rank_test_score')

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70


In [17]:
# 위 표에서 rank_test_score 의 칼럼 값은 해당 하이퍼 파라미터로 평가한 예측 성능이 1위라는 의미

In [18]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_) # 최고 성능의 하이퍼 파라미터의 값
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_)) # 그 때의 평과 결과

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [19]:
# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_ # 최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습해 best_estimator_로 저장한 것

#GridSearchCV의 best_estimator_는 이미 최적 학습이 되었으므로 별도 학습이 필요 없음
from sklearn.metrics import accuracy_score
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도:{0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도:0.9667
